In [2]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 41.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pyli

In [ ]:
import pandas as pd
import numpy as np
import collections
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, DefaultDataCollator
import evaluate
import torch
import google.generativeai as genai
from tqdm.auto import tqdm
from transformers import pipeline


2025-12-13 02:11:05.628764: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765591865.842787      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765591865.910169      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [ ]:
model_path = "/kaggle/input/finetune-bert/bert-legal-qa/checkpoint-6795"

model = AutoModelForQuestionAnswering.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

GOOGLE_API_KEY = ""
genai.configure(api_key=GOOGLE_API_KEY)
gemini_model = genai.GenerativeModel('gemini-2.5-flash')


max_length = 512    
doc_stride = 128    

In [ ]:
def find_start_index(row):
    return row['context'].find(row['answer'])

def prepare_train_features(examples):
    examples["question"] = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        examples["question"],
        examples["context"],
        max_length=max_length,
        stride=doc_stride,
        truncation="only_second",
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    sample_mapping = inputs.pop("overflow_to_sample_mapping")
    offset_mapping = inputs.pop("offset_mapping")
    inputs["start_positions"] = []
    inputs["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        sample_index = sample_mapping[i]
        answers = examples["answer"][sample_index]
        start_char = examples["answer_start"][sample_index]
        end_char = start_char + len(answers)
        sequence_ids = inputs.sequence_ids(i)

        idx = 0
        while sequence_ids[idx] != 1: idx += 1
        context_start = idx
        while sequence_ids[idx] == 1: idx += 1
        context_end = idx - 1

        if offsets[context_start][0] > start_char or offsets[context_end][1] < end_char:
            inputs["start_positions"].append(0)
            inputs["end_positions"].append(0)
        else:
            idx = context_start
            while idx <= context_end and offsets[idx][0] <= start_char: idx += 1
            inputs["start_positions"].append(idx - 1)
            idx = context_end
            while idx >= context_start and offsets[idx][1] >= end_char: idx -= 1
            inputs["end_positions"].append(idx + 1)
    return inputs

def prepare_validation_features(examples):
    examples["question"] = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        examples["question"],
        examples["context"],
        max_length=max_length,
        stride=doc_stride,
        truncation="only_second",
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = inputs.pop("overflow_to_sample_mapping")
    
    inputs["example_id"] = []

    for i in range(len(inputs["input_ids"])):
        sample_index = sample_mapping[i]
        inputs["example_id"].append(examples["id"][sample_index])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    return inputs

In [ ]:
df = pd.read_csv("/kaggle/input/data.csv")
df['id'] = df.index.astype(str)
df.head()


,Unnamed: 0,context,question,answer,id
0,0,"Tôi xin hỏi, một xã giai đoạn 2016-2020 là xã ...",Người dân tộc thiểu số xã nông thôn mới có đượ...,Bộ Y tế trả lời vấn đề này như sau:Ngày 19/10/...,0
1,1,Tôi đóng BHXH từ tháng 6/2011 đến tháng 12/201...,Có được truy lĩnh chế độ thai sản?,Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...,1
2,2,"Tôi xin hỏi, trên giấy ra viện có ghi chú ngườ...",Khi nào cần xin thêm Giấy chứng nhận nghỉ việc...,Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...,2
3,3,"Tôi năm nay 53 tuổi, đóng BHXH tự nguyện được ...",Đóng BHXH tự nguyện bao lâu thì được lương hưu?,Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...,3
4,4,Tôi đóng BHXH từ tháng 1/2020 đến tháng 12/202...,Đóng BHXH bao lâu trước khi sinh thì được hưởn...,Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...,4


In [ ]:
df['answer_start'] = df.apply(find_start_index, axis=1)
raw_datasets = Dataset.from_pandas(df)

raw_datasets = raw_datasets.train_test_split(test_size=0.2)


In [9]:
train_dataset = raw_datasets["train"].map(
    prepare_train_features,
    batched=True,
    remove_columns=raw_datasets["train"].column_names
)

validation_dataset = raw_datasets["test"].map(
    prepare_train_features,
    batched=True,
    remove_columns=raw_datasets["test"].column_names
)
predict_validation_dataset= raw_datasets["test"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=raw_datasets["test"].column_names
)

Map:   0%|          | 0/25860 [00:00<?, ? examples/s]

Map:   0%|          | 0/6465 [00:00<?, ? examples/s]

Map:   0%|          | 0/6465 [00:00<?, ? examples/s]

In [ ]:
def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size=20, max_answer_length=50):
    all_start_logits, all_end_logits = raw_predictions
    
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    
    for i, feature in enumerate(features):
        features_per_example[feature["example_id"]].append(i)

    predictions = collections.OrderedDict()

    print("--- Đang xử lý hậu kỳ (Post-processing) ---")
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        feature_indices = features_per_example[example_id]

        for feature_index in feature_indices:
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            offset_mapping = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()

            for start_index in start_indexes:
                for end_index in end_indexes:
                    if offset_mapping[start_index] is None or offset_mapping[end_index] is None:
                        continue
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    answers.append({
                        "text": context[offset_mapping[start_index][0] : offset_mapping[end_index][1]],
                        "score": start_logits[start_index] + end_logits[end_index],
                    })

        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["score"])
            predictions[example_id] = best_answer["text"]
        else:
            predictions[example_id] = "" 
    return predictions


In [ ]:
n_best_size = 180
max_length_answer = 2000

args = TrainingArguments(
    output_dir="/tmp",       
    per_device_eval_batch_size=16, 
    report_to="none",       
)


trainer = Trainer(
    model=model,
    args=args,
    data_collator=DefaultDataCollator(),
    tokenizer=tokenizer
)


raw_predictions = trainer.predict(predict_validation_dataset)

final_predictions = postprocess_qa_predictions(
    raw_datasets["test"],
    predict_validation_dataset,
    raw_predictions.predictions,
    n_best_size,
    max_length_answer
)

metric = evaluate.load("squad")

formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
formatted_references = [{"id": ex["id"], "answers": {'text': [ex["answer"]], 'answer_start': [ex["answer_start"]]}} for ex in raw_datasets["test"]]

metrics = metric.compute(predictions=formatted_predictions, references=formatted_references)
print("\nKẾT QUẢ ĐÁNH GIÁ:", metrics)


/tmp/ipykernel_47/916565218.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


--- Đang xử lý hậu kỳ (Post-processing) ---


  0%|          | 0/6465 [00:00<?, ?it/s]


KẾT QUẢ ĐÁNH GIÁ: {'exact_match': 73.08584686774942, 'f1': 82.39350378278846}


In [ ]:


qa_pipeline = pipeline(
    "question-answering",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1 
    )

def get_legal_answer(question, legal_text):

    res = qa_pipeline(
        question=question,
        context=legal_text,
        handle_impossible_answer=False,
        
        max_answer_len=180,   
        max_seq_len=512,      
        doc_stride=128,       
        truncation="only_second",
        top_k=1 
    )
    return res

question = "Người dân tộc thiểu số xã nông thôn mới có được hỗ trợ BHYT?"
documents = "Bộ Y tế trả lời vấn đề này như sau:Ngày 19/10/2023, Chính phủ ban hành Nghị định số75/2023/NĐ-CPsửa đổi, bổ sung một số điều của Nghị định số 146/2023/NĐ-CP ngày 17/10/2018 của Chính phủ quy định chi tiết và hướng dẫn biện pháp thi hành một số điều của Luật BHYT, trong đó có quy định:Người dân tộc thiểu số đang sinh sống tại địa bàn các xã khu vực II, khu vực III, thôn đặc biệt khó khăn thuộc vùng đồng bào dân tộc thiểu số và miền núi giai đoạn 2016-2020 mà các xã này không còn trong danh sách các xã khu vực II, khu vực III, thôn đặc biệt khó khăn thuộc vùng đồng bào dân tộc thiểu số và miền núi giai đoạn 2021-2025 theo quyết định của Thủ tướng Chính phủ tiếp tục được ngân sách Nhà nước hỗ trợ tối thiểu 70% mức đóng BHYT trong thời gian 36 tháng kể từ ngày 1/11/2023."
result = get_legal_answer(question, documents)
print(f"Câu hỏi: {question}")
print(f"Trả lời: {result['answer']}")
print(f"Score: {result['score']}")

Device set to use cuda:0


Câu hỏi: Người dân tộc thiểu số xã nông thôn mới có được hỗ trợ BHYT?
Trả lời: Người dân tộc thiểu số đang sinh sống tại địa bàn các xã khu vực II, khu vực III, thôn đặc biệt khó khăn thuộc vùng đồng bào dân tộc thiểu số và miền núi giai đoạn 2016-2020 mà các xã này không còn trong danh sách các xã khu vực II, khu vực III, thôn đặc biệt khó khăn thuộc vùng đồng bào dân tộc thiểu số và miền núi giai đoạn 2021-2025 theo quyết định của Thủ tướng Chính phủ tiếp tục được ngân sách Nhà nước hỗ trợ tối thiểu 70% mức đóng BHYT trong thời gian 36 tháng kể từ ngày 1/11/2023.
Score: 4.438914800175553e-07


In [ ]:
def get_top_k_answers(question, context, k=5):
    results = qa_pipeline(
        question=question,
        context=context,
        top_k=k, 
        handle_impossible_answer=False,
        max_answer_len=200,
        max_seq_len=512,
        doc_stride=128,
        truncation="only_second"
    )
    

    seen_answers = set()
    unique_results = []
    
    for res in results:
        ans_text = res['answer'].strip()
        if ans_text not in seen_answers and len(ans_text) > 5:
            unique_results.append(ans_text)
            seen_answers.add(ans_text)
            
    return unique_results

user_question = "Tôi xin hỏi, một xã giai đoạn 2016-2020 là xã khu vực III, sang giai đoạn 2021-2025 là xã khu vực I, đã đạt chuẩn xã nông thôn mới thì người dân tộc thiểu số trên địa bàn xã này có thuộc nhóm đối tượng được ngân sách Nhà nước hỗ trợ đóng BHYT không?"

list_documents = [
    "Bộ Y tế trả lời vấn đề này như sau:Ngày 19/10/2023, Chính phủ ban hành Nghị định số75/2023/NĐ-CPsửa đổi, bổ sung một số điều của Nghị định số 146/2023/NĐ-CP ngày 17/10/2018 của Chính phủ quy định chi tiết và hướng dẫn biện pháp thi hành một số điều của Luật BHYT, trong đó có quy định:Người dân tộc thiểu số đang sinh sống tại địa bàn các xã khu vực II, khu vực III, thôn đặc biệt khó khăn thuộc vùng đồng bào dân tộc thiểu số và miền núi giai đoạn 2016-2020 mà các xã này không còn trong danh sách các xã khu vực II, khu vực III, thôn đặc biệt khó khăn thuộc vùng đồng bào dân tộc thiểu số và miền núi giai đoạn 2021-2025 theo quyết định của Thủ tướng Chính phủ tiếp tục được ngân sách Nhà nước hỗ trợ tối thiểu 70% mức đóng BHYT trong thời gian 36 tháng kể từ ngày 1/11/2023.", 
    "Bảo hiểm xã hội Việt Nam trả lời vấn đề này như sau:Tại Khoản 2 Điều 28Luật BHXH năm 2006quy định điều kiện hưởng chế độ thai sản đối với lao động nữ sinh con là: Phải đóng BHXH từ đủ 6 tháng trở lên trong thời gian 12 tháng trước khi sinh con.Đối chiếu quy định nêu trên, trường hợp của bà Ngô Đảm sinh con vào tháng 8/2014, trong thời gian 12 tháng trước khi sinh con (được xác định từ tháng 8/2013 đến tháng 7/2014) bà có 12 tháng đóng BHXH (nếu đóng BHXH liên tục trong khoảng thời gian này), bà đủ điều kiện hưởng chế độ thai sản khi sinh con.Đề nghị bà liên hệ với cơ quan BHXH nơi đơn vị đã đóng BHXH để cơ quan BHXH hướng dẫn, trả lời cụ thể về việc lập hồ sơ, giải quyết chế độ thai sản đối với bà.",
]

all_extracted_infos = []

print("Đang trích xuất từ văn bản dài...")

for doc in list_documents:
    top_answers = get_top_k_answers(user_question, doc, k=5)
    top_answers_concat = "".join(top_answers)
    all_extracted_infos.append(top_answers_concat)
combined_context = "\n- ".join(all_extracted_infos)

if not combined_context:
    print("Không tìm thấy thông tin nào.")
else:
    prompt = f"""
    Dựa trên thông tin trích xuất sau đây, hãy trả lời câu hỏi: "{user_question}"
    
    THÔNG TIN:
    {combined_context}
    """
    
    print("Đang gọi Gemini tạo câu trả lời...")
    print(f"Câu hỏi : {user_question}")
    response = gemini_model.generate_content(prompt)
    print("\nCÂU TRẢ LỜI CUỐI CÙNG:\n", response.text)

Đang trích xuất từ văn bản dài...
Đang gọi Gemini tạo câu trả lời...
Câu hỏi : Tôi xin hỏi, một xã giai đoạn 2016-2020 là xã khu vực III, sang giai đoạn 2021-2025 là xã khu vực I, đã đạt chuẩn xã nông thôn mới thì người dân tộc thiểu số trên địa bàn xã này có thuộc nhóm đối tượng được ngân sách Nhà nước hỗ trợ đóng BHYT không?

CÂU TRẢ LỜI CUỐI CÙNG:
 Có.

Dựa trên thông tin đã trích xuất, người dân tộc thiểu số trên địa bàn xã này **thuộc nhóm đối tượng được ngân sách Nhà nước hỗ trợ đóng BHYT**.

**Lý do:**
Thông tin quy định rõ: "Người dân tộc thiểu số đang sinh sống tại địa bàn các xã khu vực II, khu vực III, thôn đặc biệt khó khăn thuộc vùng đồng bào dân tộc thiểu số và miền núi giai đoạn 2016-2020 mà các xã này không còn trong danh sách các xã khu vực II, khu vực III, thôn đặc biệt khó khăn thuộc vùng đồng bào dân tộc thiểu số và miền núi giai đoạn 2021-2025 theo quyết định của Thủ tướng Chính phủ tiếp tục được ngân sách Nhà nước hỗ trợ tối thiểu 70% mức đóng BHYT trong thời gian